In [1]:
!pip install mysql-connector-python

In [2]:
!pip install python-dotenv

In [3]:
import os
import mysql.connector
from mysql.connector import errorcode
from dotenv import load_dotenv

load_dotenv()

DB_CONFIG = {
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASS'),
    'host': os.getenv('DB_HOST'),
    'allow_local_infile': True,
#     'raise_on_warnings': True
}


DB_NAME = 'football'
SCHEMA_PATH = '../assets/FootballDatabase.schema.sql'
DATA_PATH = '../data/'

In [4]:
def mysql_connect(config):
    try:
        cnx = mysql.connector.connect(**config)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print(f'[CONNECTION ERROR] Invalid credentials: {err}')
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print(f'[CONNECTION ERROR] Database does not exist: {err}')
        else:
            print(f'[CONNECTION ERROR] {err}')
        return (None, None)
    else:
        cursor = cnx.cursor()
        return (cnx, cursor)
    
def mysql_close_connection(cnx):
    try:
        cnx.close()
    except mysql.connector.Error as err:
        print(f'[CLOSECONNECTION ERROR] Failes closing connection to mysql: {err}')
        raise

In [5]:
def create_database(cursor, dbname):
    try:
        cursor.execute(f'DROP DATABASE IF EXISTS {dbname}')
        cursor.execute(f'CREATE DATABASE {dbname} DEFAULT CHARACTER SET "UTF8MB4"')
    except mysql.connector.Error as err:
        print(f'[CREATEDB ERROR] Failed creating database {dbname}: {err}')
        raise

In [6]:
import re

def tables_from_schema(schema_path):
    with open(schema_path, 'r') as schema_file:
        schema = schema_file.read()
    
        # Split each `CREATE TABLE ...` in schema as an element of create_table array
        create_table = re.split('(?<!^)\s+(?=(CREATE))', schema)[::2]
    
        # Creata a dictionary with table name as key and create table script as value
        # ==> table['league'] = 'CREATE TABLE `league` ( ...'
        tables = {re.search(r'(?<=`)\w+', table)[0]: table for table in create_table}
    
        schema_file.close()
        return tables

In [7]:
def create_schema(cnx, cursor, dbname, schema):
    try:
        cursor.execute(f'USE {dbname}')
        print(f'Using database {dbname}')
    except mysql.connector.Error as err:
        print(f'Database {dbname} does not exists')
        if err.errno == errorcode.ER_BAD_DB_ERROR:
            create_database(cursor, dbname)
            print(f'Database {dbname} created successfully')
            cnx.database = dbname
        else:
            print(f'[CREATE SCHEMA ERROR] Failed creating schema: {err}')
            raise
    
    tables = tables_from_schema(schema)
    for table_name in tables:
        table_description = tables[table_name]
        try:
            print(f'Creating table {table_name}: ', end='')
            cursor.execute('SET FOREIGN_KEY_CHECKS=0')
            cursor.execute(f'DROP TABLE IF EXISTS {table_name}')
            cursor.execute('SET FOREIGN_KEY_CHECKS=1')
            cursor.execute(table_description)
        except mysql.connector.Error as err:
            print(f'[CREATE TABLE ERROR] Failed creating table: {err}')
            raise
        else:
            print('OK')
            
    return tables

In [8]:
import os

def load_data(cnx, cursor, dbname, tables, data_folder):
    cnx.raise_on_warnings = True
    cursor.execute(f'USE {dbname}')
    for table_name in tables:
        print(f'Loading data in table {table_name}')
        cursor.execute(
            f'''
            LOAD DATA LOCAL INFILE "{data_folder}/{table_name}s.csv"
            INTO TABLE {table_name}
            FIELDS TERMINATED BY ',' 
            ENCLOSED BY '"'
            LINES TERMINATED BY '\n'
            IGNORE 1 ROWS;
            '''
        )
        cnx.commit()

In [9]:
(cnx, cursor) = mysql_connect(DB_CONFIG)
tables = create_schema(cnx, cursor, DB_NAME, SCHEMA_PATH)

load_data(cnx, cursor, DB_NAME, tables, DATA_PATH)

mysql_close_connection(cnx)

Using database football
Creating table player: OK
Creating table team: OK
Creating table league: OK
Creating table game: OK
Creating table appearance: OK
Creating table teamstat: OK
Creating table shot: OK
Loading data in table player
Loading data in table team
Loading data in table league
Loading data in table game


DatabaseError: 1265: Data truncated for column 'BWH' at row 159